In [68]:
from modules.tweet_data import read_raw_data
from modules.topics import TopicSeries, display_components

import datetime as dt
import pandas as pd
import spacy
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

import matplotlib.pyplot as plt

In [46]:
tweet_df = read_raw_data()
es = pd.read_csv('data/ES.csv')

In [3]:
es = es[es.Time == '15:00'][['Date','Time','Close']]
es.Date = pd.to_datetime(es.Date +' ' +'15:45')
es.drop('Time',axis=1,inplace=True)
es.set_index('Date', inplace=True)
date_range = es['2018-6-1':'2020-5-31'].index

Reading data


In [49]:
tweet_df[date_range[0]:(date_range[1]-dt.timedelta(seconds=1))]

,tweet_id,handle,tweet
timestamp,,,
2018-06-01 15:45:03,1002576770879643649,AJEnglish,Wife of ex-Malaysian PM summoned in probe over...
2018-06-01 15:45:08,1002576790190247937,michaelsantoli,It's the collective action that matters and in...
2018-06-01 15:45:12,1002576807302975489,asymmetricinfo,"But you know, at 20, my heroes were HL Mencken..."
2018-06-01 15:45:19,1002576838542184448,ecb,It’s our 20th birthday today! Want to know mor...
2018-06-01 15:45:21,1002576844074385408,soledadobrien,*A funeral. I cannot type.
...,...,...,...
2018-06-04 15:44:40,1003663838259138560,ftchina,The unfilled vacancy for benign hegemon of glo...
2018-06-04 15:44:44,1003663854893613057,canuck2usa,$AMD so much call buying last month or so smel...
2018-06-04 15:44:45,1003663859171852288,CharlesMichel,"Bezoek OCMW van Etterbeek, ontmoeting met maat..."


In [63]:
ts=TopicSeries()
for i in range(len(date_range)-1):
    str_date = str(date_range[i].date()
    print(str_date)
    ts.calculate_nmf(str_date), 
                     tweet_df[date_range[0]:(date_range[1]-dt.timedelta(seconds=1))].tweet)

In [5]:
tweet_df = tweet_df[tweet_df.index < '06-01-2020']
single_day = tweet_df['2016-11-09']
ts =TopicSeries()
ts.calculate_day('2016-11-09', tweet_df['2016-11-09'].tweet)
ts.calculate_day('2016-05-08', tweet_df['2016-05-08'].tweet)

In [11]:
sub = [lda, cv, nmf, tfidf]

In [4]:
import pickle
pickle.dump(ts, open( "save.p", "wb" ))

In [5]:
import pickle
a = pickle.load(open( "save.p", "rb" ))

In [8]:
a.models['2016-11-09']

[LatentDirichletAllocation(n_components=5, random_state=42),
 CountVectorizer(tokenizer=<function TopicSeries.twitter_tokenizer at 0x7ffe7bc88440>),
 NMF(n_components=5, random_state=5),
 TfidfVectorizer(tokenizer=<function TopicSeries.twitter_tokenizer at 0x7ffe7bc88440>)]

In [12]:
display_components(a.models['2016-05-08'][2], a.models['2016-05-08'][3].get_feature_names(), top_display=20)

Topic 0:
day happy mother mothers mom moms love #mothersday great thank wife nice time celebrating appreciation important today life enjoy world
Topic 1:
trump donald says clinton hillary gop party women republican ryan going president win taxes paul nominee wealthy better candidate said
Topic 2:
lol yep seen look actual idiot irl pic pro true picks trying fireworks nails choice texas nice huh answer haha
Topic 3:
good luck morning point love story man pretty night new nt book true time years listen wo saudi change add
Topic 4:
like know thanks think people yes new look right sure going time saudi oh great actually minister china oil looks
